# Tweet Classification

### Import Libraries

In [1]:
import pandas as pd
import numpy as np
import os
import sys
from textprep import text_prep
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.models import model_from_json
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Input
from tensorflow.keras.utils import to_categorical
from sklearn import preprocessing
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score,f1_score,confusion_matrix
from tensorflow.keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation,Lambda
from gensim.models import FastText
import json
import pickle
from tensorflow.python.framework import ops
ops.reset_default_graph()

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/ssabu2/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/ssabu2/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /Users/ssabu2/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /Users/ssabu2/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### Load Twitter Data

In [2]:
dataset1 = pd.read_excel("tweets3.xlsx")
print(dataset1.shape)
dataset1.head()

(852, 5)


,Unnamed: 0,Tweet_ID,GT,Text,Created_At
0,0.0,1.317271e+18,Y,left my glasses at a restaurant and called the...,Sat Oct 17 01:05:47 +0000 2020
1,1.0,1.317270e+18,N,I worked at a restaurant that was requiring th...,Sat Oct 17 01:03:17 +0000 2020
2,2.0,1.317269e+18,Y,"@testpatternshow Went to a local restaurant, &...",Sat Oct 17 00:58:38 +0000 2020
3,3.0,1.317269e+18,Y,OMG...went to pick up food at a restaurant tha...,Sat Oct 17 00:57:43 +0000 2020
4,4.0,1.317266e+18,Y,Update from the Inner Cook County Resistance: ...,Sat Oct 17 00:49:17 +0000 2020


In [3]:
dataset2 = pd.read_excel("tweets4.xlsx")
print(dataset2.shape)
dataset2.head()

(1076, 5)


,Unnamed: 0,Tweet_ID,GT,Text,Created_At
0,0.0,1.323697e+18,N,I do!!! When was the last time you went to an ...,Tue Nov 03 18:41:49 +0000 2020
1,1.0,1.323689e+18,N,Hoped off the plane from Vermont the 18th of O...,Tue Nov 03 18:09:30 +0000 2020
2,2.0,1.323677e+18,N,@runmyheart I once went to an art event where ...,Tue Nov 03 17:21:59 +0000 2020
3,3.0,1.323670e+18,N,@AndHeGrows I went 30-11 on a juggernaut round...,Tue Nov 03 16:56:04 +0000 2020
4,4.0,1.323661e+18,N,2016 I remember myself and some friends making...,Tue Nov 03 16:19:24 +0000 2020


In [4]:
dataset=pd.concat([dataset1,dataset2],axis=0)

In [5]:
dataset.shape

(1928, 5)

In [6]:
dataset.head()

,Unnamed: 0,Tweet_ID,GT,Text,Created_At
0,0.0,1.317271e+18,Y,left my glasses at a restaurant and called the...,Sat Oct 17 01:05:47 +0000 2020
1,1.0,1.317270e+18,N,I worked at a restaurant that was requiring th...,Sat Oct 17 01:03:17 +0000 2020
2,2.0,1.317269e+18,Y,"@testpatternshow Went to a local restaurant, &...",Sat Oct 17 00:58:38 +0000 2020
3,3.0,1.317269e+18,Y,OMG...went to pick up food at a restaurant tha...,Sat Oct 17 00:57:43 +0000 2020
4,4.0,1.317266e+18,Y,Update from the Inner Cook County Resistance: ...,Sat Oct 17 00:49:17 +0000 2020


### Pre-process Tweet

In [7]:
dataset=dataset[['Text','GT']]
dataset.columns=['sentence','label']
dataset.head()

,sentence,label
0,left my glasses at a restaurant and called the...,Y
1,I worked at a restaurant that was requiring th...,N
2,"@testpatternshow Went to a local restaurant, &...",Y
3,OMG...went to pick up food at a restaurant tha...,Y
4,Update from the Inner Cook County Resistance: ...,Y


In [8]:
dataset.sentence = dataset.sentence.apply(lambda x: text_prep(x))

In [9]:
dataset.dropna(inplace=True, axis=0)

In [10]:
dataset['label']=dataset['label'].apply(lambda x: str(x).lower())

In [11]:
dataset.shape

(1314, 2)

In [12]:
dataset['label'].unique()

array(['y', 'n'], dtype=object)

In [13]:
dataset = dataset.sample(frac = 1) 
dataset.shape

(1314, 2)

### Train-Test Split

In [14]:
dataset_train=dataset[:1100]
dataset_test=dataset[1100:1314]

## Method 1 - Using TF-IDF

In [15]:
sent=dataset_train['sentence'].tolist()

tfidf_vectorizer = TfidfVectorizer()
x_train = tfidf_vectorizer.fit_transform(sent).toarray()

pickle.dump(tfidf_vectorizer, open('tfidf_vectorizer.pkl', 'wb'))

y_train = list(dataset_train['label'])
le = preprocessing.LabelEncoder()
le.fit(y_train)

pickle.dump(le, open('LabelEncoder.pkl', 'wb'))
def encode(le, labels):
    enc = le.transform(labels)
    return to_categorical(enc)

def decode(le, one_hot):
    dec = np.argmax(one_hot, axis=1)
    return le.inverse_transform(dec)

y_train = encode(le, y_train)
y_train = np.asarray(y_train)

In [16]:
y_train

array([[1., 0.],
       [1., 0.],
       [0., 1.],
       ...,
       [1., 0.],
       [0., 1.],
       [0., 1.]], dtype=float32)

### Model Building

In [19]:
model = Sequential()
model.add(Dense(128, activation='sigmoid',input_shape=(5779,)))
model.add(Dropout(0.5))
model.add(Dense(64, activation='sigmoid'))
model.add(Dropout(0.3))
model.add(Dense(2, activation="softmax"))
model.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])

In [20]:
history = model.fit(x_train, y_train, batch_size = 32, epochs = 20, validation_split = .2)

Epoch 1/20
28/28 [==============================] - 0s 7ms/step - loss: 0.9556 - accuracy: 0.4898 - val_loss: 0.7455 - val_accuracy: 0.5273
Epoch 2/20
28/28 [==============================] - 0s 3ms/step - loss: 0.7469 - accuracy: 0.5057 - val_loss: 0.6898 - val_accuracy: 0.5273
Epoch 3/20
28/28 [==============================] - 0s 3ms/step - loss: 0.7277 - accuracy: 0.5080 - val_loss: 0.6916 - val_accuracy: 0.5273
Epoch 4/20
28/28 [==============================] - 0s 4ms/step - loss: 0.7251 - accuracy: 0.4977 - val_loss: 0.6889 - val_accuracy: 0.5273
Epoch 5/20
28/28 [==============================] - 0s 3ms/step - loss: 0.7045 - accuracy: 0.5386 - val_loss: 0.6876 - val_accuracy: 0.5409
Epoch 6/20
28/28 [==============================] - 0s 4ms/step - loss: 0.7038 - accuracy: 0.5250 - val_loss: 0.6909 - val_accuracy: 0.5273
Epoch 7/20
28/28 [==============================] - 0s 3ms/step - loss: 0.7074 - accuracy: 0.5273 - val_loss: 0.6864 - val_accuracy: 0.5273
Epoch 8/20
28/28 [==

### Save Model

In [21]:

model_json = model.to_json()
with open("tfidfmodel.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights("tfidf_model.h5")

print("Training Done")
print("Model Saved")

Training Done
Model Saved


In [22]:
pickle_in = open('tfidf_vectorizer.pkl',"rb")
tfidf_vectorizer = pickle.load(pickle_in)
sent=dataset_test['sentence'].tolist()
x_test = tfidf_vectorizer.transform(sent).toarray()

### Load Model

In [23]:
y_test = list(dataset_test['label'])
pickle_in = open('LabelEncoder.pkl',"rb")
le_pickle = pickle.load(pickle_in)
y_test = encode(le_pickle, y_test)
y_test = np.asarray(y_test)

json_file = open("tfidfmodel.json", 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
loaded_model.load_weights("tfidf_model.h5")


In [24]:
predicts = loaded_model.predict(x_test, batch_size=32,verbose=2)

7/7 - 0s


In [25]:
y_preds = decode(le_pickle, predicts)


In [26]:
y_test = decode(le_pickle, y_test)

### Evaluation

In [27]:
print("Accuracy : ",accuracy_score(y_test, y_preds))
print("F1 Score : ",f1_score(y_test, y_preds,average='weighted'))
print("confusion_matrix : ",confusion_matrix(y_test, y_preds))

Accuracy :  0.7102803738317757
F1 Score :  0.6839953271028038
confusion_matrix :  [[ 31  50]
 [ 12 121]]


## Method 2 - Using word2vec

In [28]:
embeddings_index={}
#reading embeddings
with open('glove.6B.200d.txt','r',encoding='utf-8') as embd_file:
    for line in embd_file:
        val = line.split()
        word = val[0]
        cfs = np.asarray(val[1:], dtype='float32')
        embeddings_index[word] = cfs

pickle.dump(embeddings_index, open('embeddings_index.pkl', 'wb'))

def avg_vectors(sentence):
    sentence = [token for token in sentence.split() if  token in embeddings_index]
    embedding_size = 200
    vs = np.zeros(embedding_size)
    sentence_length = len(sentence)
    for word in sentence:
        vs = np.add(vs, embeddings_index.get(word))
    vs = np.divide(vs, sentence_length)
    return vs

In [29]:
x_train_vec= dataset_train.sentence.apply(lambda x: avg_vectors(x))
x_train=np.array(x_train_vec.values.tolist())

y_train = list(dataset_train['label'])
le = preprocessing.LabelEncoder()
le.fit(y_train)
pickle.dump(le, open('LabelEncoder.pkl', 'wb'))
def encode(le, labels):
    enc = le.transform(labels)
    return to_categorical(enc)

def decode(le, one_hot):
    dec = np.argmax(one_hot, axis=1)
    return le.inverse_transform(dec)

y_train = encode(le, y_train)
y_train = np.asarray(y_train)

In [30]:
model_w2v = Sequential()
model_w2v.add(Dense(128, activation='sigmoid',input_shape=(200,)))
model_w2v.add(Dropout(0.3))
model_w2v.add(Dense(64, activation='sigmoid'))
model_w2v.add(Dropout(0.3))
model_w2v.add(Dense(2, activation="softmax"))
model_w2v.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])

model_w2v.fit(x_train, y_train, batch_size = 32, epochs = 15, validation_split = .2)

Epoch 1/15
28/28 [==============================] - 0s 5ms/step - loss: 0.7529 - accuracy: 0.5125 - val_loss: 0.6884 - val_accuracy: 0.5045
Epoch 2/15
28/28 [==============================] - 0s 1ms/step - loss: 0.7441 - accuracy: 0.4886 - val_loss: 0.6775 - val_accuracy: 0.5818
Epoch 3/15
28/28 [==============================] - 0s 1ms/step - loss: 0.7123 - accuracy: 0.5318 - val_loss: 0.6802 - val_accuracy: 0.5318
Epoch 4/15
28/28 [==============================] - 0s 2ms/step - loss: 0.7001 - accuracy: 0.5386 - val_loss: 0.6717 - val_accuracy: 0.5773
Epoch 5/15
28/28 [==============================] - 0s 1ms/step - loss: 0.6872 - accuracy: 0.5670 - val_loss: 0.6603 - val_accuracy: 0.6136
Epoch 6/15
28/28 [==============================] - 0s 1ms/step - loss: 0.6795 - accuracy: 0.5716 - val_loss: 0.6550 - val_accuracy: 0.6227
Epoch 7/15
28/28 [==============================] - 0s 1ms/step - loss: 0.6588 - accuracy: 0.6261 - val_loss: 0.6555 - val_accuracy: 0.6000
Epoch 8/15
28/28 [==

### Save Model

In [31]:
model_json = model_w2v.to_json()
with open("word2vecmodel.json", "w") as json_file:
    json_file.write(model_json)
model_w2v.save_weights("word2vec_model.h5")

print("Training Done")
print("Model Saved")

Training Done
Model Saved


In [32]:

pickle_in = open('embeddings_index.pkl',"rb")
embeddings_index = pickle.load(pickle_in)

def avg_vectors(sentence):
    sentence = [token for token in sentence.split() if  token in embeddings_index]
    embedding_size = 200
    vs = np.zeros(embedding_size)
    sentence_length = len(sentence)
    for word in sentence:
        vs = np.add(vs, embeddings_index.get(word))
    vs = np.divide(vs, sentence_length)
    return vs

x_test_vec= dataset_test.sentence.apply(lambda x: avg_vectors(x))
x_test=np.array(x_test_vec.values.tolist())


def encode(le, labels):
    enc = le.transform(labels)
    return to_categorical(enc)

def decode(le, one_hot):
    dec = np.argmax(one_hot, axis=1)
    return le.inverse_transform(dec)

y_test = list(dataset_test['label'])
pickle_in = open('LabelEncoder.pkl',"rb")
le_pickle = pickle.load(pickle_in)
y_test = encode(le_pickle, y_test)
y_test = np.asarray(y_test)

json_file_w2v = open("word2vecmodel.json", 'r')
loaded_model_json_w2v = json_file_w2v.read()
json_file_w2v.close()
loaded_model_w2v = model_from_json(loaded_model_json_w2v)
loaded_model_w2v.load_weights("word2vec_model.h5")


### Evaluation

In [33]:
predict = loaded_model_w2v.predict(x_test, batch_size=32,verbose=2)
y_test = decode(le_pickle, y_test)
y_preds = decode(le_pickle, predict)
print("Accuracy : ",accuracy_score(y_test, y_preds))
print("F1 Score : ",f1_score(y_test, y_preds,average='weighted'))
print("confusion_matrix : ",confusion_matrix(y_test, y_preds))

print("Testing Done")
print("Output Created")

7/7 - 0s
Accuracy :  0.5747663551401869
F1 Score :  0.5769496132412127
confusion_matrix :  [[59 22]
 [69 64]]
Testing Done
Output Created


## Method 3 - Using ELMO

In [34]:
import tensorflow.compat.v1 as tf
tf.disable_eager_execution()
import tensorflow_hub as hub

In [35]:
print(tf.__version__)
print(hub.__version__)

2.3.1
0.10.0


In [36]:
y_train = list(dataset_train['label'])
x_train = list(dataset_train['sentence'])

lb = preprocessing.LabelEncoder()
lb.fit(y_train)
pickle.dump(lb, open('LabelEncoder.pkl', 'wb'))

def encd(lb, labels):
    enc = lb.transform(labels)
    return to_categorical(enc)

def decd(lb, one_hot):
    dec = np.argmax(one_hot, axis=1)
    return lb.inverse_transform(dec)

y_train = encd(lb, y_train)

x_train = np.asarray(x_train)
y_train = np.asarray(y_train)

In [37]:
url = "https://tfhub.dev/google/elmo/2"
embed = hub.Module(url)

def elmoembd(x):
    return embed(tf.squeeze(tf.cast(x, tf.string)), signature="default", as_dict=True)["default"]

In [38]:
input_text = Input(shape=(1,), dtype=tf.string)
embedding = Lambda(elmoembd, output_shape=(1024, ))(input_text)
dense1 = Dense(128, activation='relu')(embedding)
dense2= Dropout(0.5)(dense1)
dense3 = Dense(256, activation='relu')(dense2)
dense4= Dropout(0.3)(dense3)
pred = Dense(2, activation='softmax')(dense4)
model = Model(inputs=[input_text], outputs=pred)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [39]:
with tf.Session() as session:
    tf.compat.v1.keras.backend.set_session(session)
    session.run(tf.global_variables_initializer())
    session.run(tf.tables_initializer())
    history = model.fit(x_train, y_train, epochs=5, batch_size=32,verbose=1, validation_split=0.2)
    model_json = model.to_json()
    with open("elmomodel.json", "w") as json_file:
        json_file.write(model_json)
    model.save_weights("elmo_model.h5")

print("Training Done")
print("Model Saved")

Train on 880 samples, validate on 220 samples
Epoch 1/5
880/880 [==============================] - ETA: 0s - loss: 0.7076 - accuracy: 0.5330WARNING:tensorflow:From /Users/ssabu2/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/keras/engine/training_v1.py:2048: Model.state_updates (from tensorflow.python.keras.engine.training) is deprecated and will be removed in a future version.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


880/880 [==============================] - 64s 73ms/sample - loss: 0.7076 - accuracy: 0.5330 - val_loss: 0.6600 - val_accuracy: 0.6091
Epoch 2/5
880/880 [==============================] - 66s 75ms/sample - loss: 0.6855 - accuracy: 0.5568 - val_loss: 0.6456 - val_accuracy: 0.6136
Epoch 3/5
880/880 [==============================] - 60s 68ms/sample - loss: 0.6256 - accuracy: 0.6500 - val_loss: 0.6284 - val_accuracy: 0.6227
Epoch 4/5
880/880 [==============================] - 58s 66ms/sample - loss: 0.6134 - accuracy: 0.6636 - val_loss: 0.6308 - val_accuracy: 0.6364
Epoch 5/5
880/880 [==============================] - 60s 68ms/sample - loss: 0.5927 - accuracy: 0.6932 - val_loss: 0.6239 - val_accuracy: 0.6500
Training Done
Model Saved


In [40]:
y_test = list(dataset_test['label'])
x_test = list(dataset_test['sentence'])

pickle_in = open('LabelEncoder.pkl',"rb")
le_pickle = pickle.load(pickle_in)

def encode(lb, labels):
    enc = lb.transform(labels)
    return to_categorical(enc)

def decode(lb, one_hot):
    dec = np.argmax(one_hot, axis=1)
    return lb.inverse_transform(dec)

y_test = encode(le_pickle, y_test)

x_test = np.asarray(x_test)
y_test = np.asarray(y_test)


### Evaluation

In [41]:
with tf.Session() as session:
    tf.compat.v1.keras.backend.set_session(session)
    session.run(tf.global_variables_initializer())
    session.run(tf.tables_initializer())
    json_file = open('elmomodel.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    loaded_model.load_weights("elmo_model.h5")
    predicts = loaded_model.predict(x_test, batch_size=32,verbose=2)
    
y_test = decode(le_pickle, y_test)
y_preds = decode(le_pickle, predicts)

print("Accuracy : ",accuracy_score(y_test, y_preds))
print("F1 Score : ",f1_score(y_test, y_preds,average='weighted'))
print("confusion_matrix : ",confusion_matrix(y_test, y_preds))


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


Accuracy :  0.6448598130841121
F1 Score :  0.6322674298190949
confusion_matrix :  [[ 33  48]
 [ 28 105]]
